## EO Data Extraction Workflow
This notebook demonstrates a streamlined workflow for extracting and processing Earth Observation (EO) data 
using the **openEO** Python client. 

### Key Steps:
1. Load and align input data (shapefile).
2. Transform GeoDataFrame for MultiBackendJobManager
3. Run the extraction process using openEO backends.
4. View and analyze the outputs (e.g., NetCDF files).

### Required Libraries:
- `openeo` for interacting with EO backends.
- `openeo-gfmap` for handling geospatial data.

### Step 1: load in the shapefile

In [ ]:
import os
import zipfile
import eotdl
from eotdl.datasets import download_dataset

download_dataset("EuroCrops", version=1, path="data", force=True)

os.makedirs("data/EuroCrops", exist_ok=True)

with zipfile.ZipFile("./data/EuroCrops/v1/EuroCrops.zip", 'r') as zip_ref:
    zip_ref.extractall("data/EuroCrops")


c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 7.91G/7.91G [18:58<00:00, 7.46MiB/s]
100%|██████████| 1/1 [18:58<00:00, 1138.85s/file]


## Extract the desired geodataframe

In [1]:
import geopandas as gpd

# Define the file path
file_path = r"C:\Git_projects\eotdl\tutorials\notebooks\openeo\data\EuroCrops\BE_VLG_2021\BE_VLG_2021_EC21.shp"

# Load the shapefile
gdf = gpd.read_file(file_path)


## Subset the dataset for testing purposes

In [3]:
gdf = gdf[0:2]
gdf

,fid,GRAF_OPP,REF_ID,GWSCOD_V,GWSNAM_V,GWSCOD_H,GWSNAM_H,GWSGRP_H,GWSGRPH_LB,GWSCOD_N,...,PRC_NIS,X_REF,Y_REF,WGS84_LG,WGS84_BG,EC_NUTS3,EC_trans_n,EC_hcat_n,EC_hcat_c,geometry
0,56.0,1.0038,2.195943e+09,None,None,898,Permacultuur,Andere subsidiabele gewassen groenten - gebrui...,Overige gewassen,None,...,31033,59203.23,191139.13,"3°4'28""","51°1'24""",BE251,permaculture,permanent_crops_perennial,3303000000,"POLYGON ((59139.6 191171.57, 59230.75 191204.2..."
1,72.0,1.7600,2.192143e+09,None,None,898,Permacultuur,Andere subsidiabele gewassen groenten - gebrui...,Overige gewassen,None,...,31040,66180.66,206540.11,"3°10'13""","51°9'46""",BE251,permaculture,permanent_crops_perennial,3303000000,"POLYGON ((66098.5 206582.75, 66141.25 206653.7..."


# Transform GeoDataFrame for MultiBackendJobManager

This function processes an input GeoDataFrame and prepares it for use with openEO's **MultiBackendJobManager**. The job manager enables launching and tracking multiple openEO jobs simultaneously, which is essential for large-scale data extractions. 

### Example Use Case
The function creates patches (e.g., 64x64 pixels) around polygon centers. These patches are suitable for machine learning applications, such as training convolutional neural networks (CNNs).  
By combining patches into Sentinel-2 grid collections, the workflow ensures cost efficiency and optimized data extraction.

### Workflow

1. **Process the GeoDataFrame**  
   - Create patches with a fixed size around the center of polygon geometries.  
   - Calculate temporal extents for each geometry.  

2. **Combine Features Using Sentinel-2 Tiling**  
   - Group buffered geometries into collections based on the Sentinel-2 tiling grid.  
   - Minimize redundant openEO cost.  

3. **Generate Job Metadata DataFrame**  
   - Convert processed data into a DataFrame, ready for the MultiBackendJobManager.

### Parameters

#### Spatial Parameters:
- **Buffer Distance:** Buffer size (e.g., 320 meters for a 64x64 patch around polygon centers).  
- **Resolution:** Spatial alignment resolution in meters.

#### Temporal Parameters:
- **Start Date:** Start of the temporal extent (e.g., `"2020-01-01"`).  
- **Number of Months:** Duration of the temporal extent in months.

#### Job Splitting Parameters:
- **Max Points Per Job:** Maximum number of features per job batch.


In [4]:
from dataframe_utils import *

# Constants
start_date = "2020-01-01"
nb_months = 3

pixel_size = 64  # Buffer distance in meters
resolution = 10

job_df = process_and_create_jobs(
    gdf, start_date, nb_months, pixel_size, resolution, max_points=1)

job_df

c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["centroid"] = polygons.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:84: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_grid["geometry"] = s2_grid.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before t

,temporal_extent,geometry,s2_tile,h3index,crs,feature_count
0,"[2020-01-01, 2020-04-01]","{""type"": ""FeatureCollection"", ""features"": [{""i...",31UES,83194dfffffffff,EPSG:32631,1
1,"[2020-01-01, 2020-04-01]","{""type"": ""FeatureCollection"", ""features"": [{""i...",31UES,83194dfffffffff,EPSG:32631,1


In [5]:
job_df.geometry.iloc[0]

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"fid": 56.0, "crs": "EPSG:32631", "temporal_extent": ["2020-01-01", "2020-04-01"], "h3index": "83194dfffffffff", "tile": "31UES"}, "geometry": {"type": "Polygon", "coordinates": [[[505550.0, 5652740.0], [505550.0, 5652099.999999999], [504910.0, 5652100.0], [504909.99999999994, 5652740.0], [505550.0, 5652740.0]]]}}], "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::32631"}}}'

# Start Job with Standardized UDPs and Feature Collection Filtering

This function initializes an openEO batch job using standardized **User-Defined Processes (UDPs)** for Sentinel-1 and Sentinel-2 data processing. It employs a spatial filter designed for non-lat/lon feature collections to ensure precise patch sizes in UTM coordinates.

### Key Features

1. **Use of Standardized UDPs**  
   - **S1 Weekly Statistics:** Computes weekly statistics from Sentinel-1 data.  
   - **S2 Weekly Statistics:** Computes weekly statistics from Sentinel-2 data.  
   - UDPs are defined in external JSON files.

2. **Spatial Filtering with `load_url`**  
   - Accepts feature collections in **UTM coordinates** to guarantee patches with exact dimensions (e.g., 64x64 meters).  
   - Features are uploaded as a GeoParquet file to an creodias S3 bucket, enabling spatial filtering directly on the server.

3. **Cube Merging**  
   - Merges Sentinel-1 and Sentinel-2 datacubes for combined analysis.

4. **Job Configuration**  
   - Saves each feature in the collection as a separate file.  
   - Outputs results in **NetCDF** format with filenames derived

In [ ]:
import geojson
import openeo
from s3proxy_utils import upload_geoparquet_file

def start_job(row: pd.Series, connection: openeo.Connection, **kwargs) -> openeo.BatchJob:

        temporal_extent = row["temporal_extent"]
        crs = row['crs']

        # set up load url in order to allow non-latlon feature collections for spatial filtering
        geometry = geojson.loads(row["geometry"])
        features = gpd.GeoDataFrame.from_features(geometry).set_crs(crs)
        url = upload_geoparquet_file(features,connection)

        #run the s1 and s2 udp
        s1 = connection.datacube_from_process(
                "s1_weekly_statistics",
                namespace="https://raw.githubusercontent.com/earthpulse/eotdl/refs/heads/hv_openeoexample/tutorials/notebooks/openeo/s1_weekly_statistics.json",
                temporal_extent=temporal_extent,
                )
        
        s2 = connection.datacube_from_process(
                "s2_weekly_statistics",
                namespace="https://raw.githubusercontent.com/earthpulse/eotdl/refs/heads/hv_openeoexample/tutorials/notebooks/openeo/s2_weekly_statistics.json",
                temporal_extent=temporal_extent,
                )
        
        #merge both cubes and filter across the feature collection
        merged = s2.merge_cubes(s1)
        result = merged.filter_spatial(connection.load_url(url, format="Parquet"))
        
        #dedicated job settings to save the individual features within a collection seperately
        job = result.create_job(
                out_format="NetCDF",
                sample_by_feature = True,
                feature_id_property = "fid",
                filename_prefix = "eotdl"
        )

        return job

### Submit Extraction Jobs

Using the openEO backend, we authenticate and submit the jobs to process the EO data. 
Each job extracts Sentinel 1 and Sentinel 2 training features.

In [7]:
import openeo
from openeo.extra.job_management import MultiBackendJobManager, CsvJobDatabase

# Authenticate and add the backend
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()

# initialize the job manager
manager = MultiBackendJobManager()
manager.add_backend("cdse", connection=connection, parallel_jobs=2)

job_tracker = 'jobs.csv'
job_db = CsvJobDatabase(path=job_tracker)
if not job_db.exists():
    df = manager._normalize_df(job_df)
    job_db.persist(df)

manager.run_jobs(df = job_df, start_job=start_job, job_db=job_db)



Authenticated using refresh token.


c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo\extra\job_management.py:394: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'cdse' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "backend_name"] = backend_name
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo\extra\job_management.py:410: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-12-04T23:09:27.260305' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "start_time"] = datetime.datetime.now().isoformat()
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo\extra\job_management.py:412: FutureWarning: Setting an item of incompatible dtype is deprecated an

Step 4: We run the various openEO Jobs. Note all data will be locally downloaded as netcdfs named after the file_name property within the individual features (see process_file)

TODO;
1) simplified example. (non cost efficient)
--> point extraction (parcel averaged)
2) integrate with eotdl example
3) export workspace
4) advanced patch extraction
5) signed URL